In [1]:
import pandas as pd
from nba_api.stats.static import players

players_act = players.get_active_players()
player_ids = [player["id"] for player in players_act]
# len(player_ids)

544

In [2]:
from nba_api.stats.endpoints import playerprofilev2

profiles = [playerprofilev2.PlayerProfileV2(player_id=id).get_data_frames()[0] for id in player_ids]
df = pd.concat(profiles)
df = df[["PLAYER_ID", "SEASON_ID", "GP", "MIN", "FGM", "FGA", "FG_PCT", "FG3M", "FG3A", "FG3_PCT"]]
df.head()

,PLAYER_ID,SEASON_ID,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT
0,1630173,2020-21,61,737,124,228,0.544,0,1,0.000
1,1630173,2021-22,73,1725,265,603,0.439,56,156,0.359
2,1630173,2022-23,55,1141,196,404,0.485,29,108,0.269
3,1630173,2023-24,25,437,78,170,0.459,13,47,0.277
4,1630173,2023-24,49,1187,157,299,0.525,13,50,0.260


In [28]:
df.to_csv("player_profiles_full.csv", index=False)

Attempt to use PlayerCareerStats endpoint. Produces same results as previous approach.

In [ ]:
# from nba_api.stats.endpoints import playercareerstats

# stats = [playercareerstats.PlayerCareerStats(player_id=id).get_data_frames()[0] for id in player_ids]
# df_stats = pd.concat(stats)
# df_stats.head()

Tried to limit player shooting percentage to last season, but every player can still have multiple items - they are split by teams (e.g. if a player played for multiple teams during season). This creates problems when joining shooting data with pbp data. Most likely necessary to aggregate this data. How??

In [ ]:
# df = df[df["SEASON_ID"] == "2024-25"]
# df = df[["PLAYER_ID", "FG_PCT", "FG3_PCT"]]
# df.shape
# df.head()

In [5]:
import pandas as pd

df_profiles = pd.read_csv("player_profiles_full.csv")
df_profiles = df_profiles[df_profiles["SEASON_ID"] == "2024-25"]
df_profiles

,PLAYER_ID,SEASON_ID,GP,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT
6,1630173,2024-25,44,927,122,235,0.519,9,30,0.300
17,203500,2024-25,49,618,70,134,0.522,0,1,0.000
25,1628389,2024-25,66,2272,444,913,0.486,54,174,0.310
30,1630534,2024-25,55,1482,222,441,0.503,83,208,0.399
34,1630583,2024-25,53,1325,263,540,0.487,96,256,0.375
...,...,...,...,...,...,...,...,...,...,...
3472,1629027,2024-25,63,2281,463,1147,0.404,173,533,0.325
3494,1627826,2024-25,66,2152,475,763,0.623,0,0,0.000
3495,1641783,2024-25,65,1481,179,429,0.417,73,218,0.335
3500,1628427,2024-25,8,67,6,12,0.500,2,5,0.400


In [6]:
idx = df_profiles.groupby("PLAYER_ID")["GP"].idxmax()

df_maxgp = df_profiles.loc[idx].reset_index(drop=True)
df_maxgp.shape

(532, 10)

In [ ]:
df_maxgp.to_csv("player_profiles24.csv", index=False)

In [7]:
df_maxgp = df_maxgp[["PLAYER_ID", "FG_PCT", "FG3_PCT"]]

START NOT NECESSARY

In [ ]:
sum_cols = ['GP', 'MIN', 'FGM', 'FGA', 'FG3M', 'FG3A']

# Group by PLAYER_ID and SEASON_ID
grouped = df_profiles.groupby(['PLAYER_ID', 'SEASON_ID'], as_index=False)
sum_df = grouped[sum_cols].sum()

# Now compute weighted FG_PCT and FG3_PCT
def weighted_pct(group):
    fg_pct = (group['FG_PCT'] * group['FGA']).sum() / group['FGA'].sum() if group['FGA'].sum() != 0 else 0
    fg3_pct = (group['FG3_PCT'] * group['FG3A']).sum() / group['FG3A'].sum() if group['FG3A'].sum() != 0 else 0
    return pd.Series({'FG_PCT': fg_pct, 'FG3_PCT': fg3_pct})

# Apply the weighted average function
weighted_df = grouped.apply(weighted_pct).reset_index()

# Merge summed and weighted data
final_df = pd.merge(sum_df, weighted_df, on=['PLAYER_ID', 'SEASON_ID'])
final_df

In [7]:
final_df = final_df[["PLAYER_ID", "FG_PCT", "FG3_PCT"]]

Simple aggregation - probably not useful.

In [ ]:
agg_dict = {
    'GP': 'sum',
    'MIN': 'sum',
    'FGM': 'sum',
    'FGA': 'sum',
    'FG3M': 'sum',
    'FG3A': 'sum',
    'FG_PCT': 'mean',
    'FG3_PCT': 'mean'
}

# Group by PLAYER_ID and SEASON_ID, then aggregate
agg_df = df.groupby(['PLAYER_ID', 'SEASON_ID'], as_index=False).agg(agg_dict)

Weighted aggregation - shooting pctg weighted by number of games.

In [ ]:
sum_cols = ['GP', 'MIN', 'FGM', 'FGA', 'FG3M', 'FG3A']

# Group by PLAYER_ID and SEASON_ID
grouped = df.groupby(['PLAYER_ID', 'SEASON_ID'], as_index=False)

# Sum the numeric stats
sum_df = grouped[sum_cols].sum()

# Now compute weighted FG_PCT and FG3_PCT
def weighted_pct(group):
    fg_pct = (group['FG_PCT'] * group['FGA']).sum() / group['FGA'].sum() if group['FGA'].sum() != 0 else 0
    fg3_pct = (group['FG3_PCT'] * group['FG3A']).sum() / group['FG3A'].sum() if group['FG3A'].sum() != 0 else 0
    return pd.Series({'FG_PCT': fg_pct, 'FG3_PCT': fg3_pct})

# Apply the weighted average function
weighted_df = grouped.apply(weighted_pct).reset_index()

# Merge summed and weighted data
final_df = pd.merge(sum_df, weighted_df, on=['PLAYER_ID', 'SEASON_ID'])


END NOT NECESSARY

In [8]:
shots_df = pd.read_csv("data_v1_ft.csv")
shots_df.head()

,gameId,actionNumber,clock,period,teamId,teamTricode,personId,playerNameI,xLegacy,yLegacy,shotDistance,shotResult,isFieldGoal,scoreHome,scoreAway,location,actionType,subType,shotValue,actionId
0,22401172,7,PT11M48.00S,1,1610612753,ORL,202709,C. Joseph,227,21,0,Missed,1,NaN,NaN,v,Missed Shot,Jump Shot,3,3
1,22401172,9,PT11M41.00S,1,1610612753,ORL,1629048,G. Bitadze,7,-11,1,Made,1,0.0,2.0,v,Made Shot,Alley Oop Dunk Shot,2,5
2,22401172,11,PT11M20.00S,1,1610612754,IND,1630167,O. Toppin,-86,46,10,Made,1,2.0,2.0,h,Made Shot,Driving Floating Bank Jump Shot,2,6
3,22401172,12,PT11M08.00S,1,1610612753,ORL,203914,G. Harris,108,256,28,Missed,1,NaN,NaN,v,Missed Shot,Jump Shot,3,7
4,22401172,14,PT10M52.00S,1,1610612754,IND,1631097,B. Mathurin,71,270,28,Missed,1,NaN,NaN,h,Missed Shot,Pullup Jump shot,3,9


Join player shooting PCT in current season with shot data.

In [ ]:
data_v2_ft = pd.merge(shots_df, df_maxgp, how="left", left_on="personId", right_on="PLAYER_ID")
data_v2_ft = data_v2_ft.drop(columns=["PLAYER_ID"])
data_v2_ft.head()

Data_v2 contains additionally player shooting pctg. This is calculated within season - could extend to multiple seasons. Would leave this depending on classification results.

In [21]:
data_v2_ft.to_csv("data_v2_ft.csv", index=False)

In [ ]:
# data_v2_ft["gameId"].unique()

###############################################################################################################################################################

In [ ]:
import numpy as np

def fill_scores(group):
    # Replace NaN in the first row with 0
    if pd.isna(group.iloc[0]['scoreHome']):
        group.iloc[0, group.columns.get_loc('scoreHome')] = 0.0
    if pd.isna(group.iloc[0]['scoreAway']):
        group.iloc[0, group.columns.get_loc('scoreAway')] = 0.0
    # Forward-fill the rest
    group['scoreHome'] = group['scoreHome'].ffill()
    group['scoreAway'] = group['scoreAway'].ffill()
    return group

# Apply the function to each gameId group
data_v3_ft = data_v2_ft.groupby('gameId', group_keys=False).apply(fill_scores)

# game2 = test_res[test_res['gameId'] == 22400102]

# print(game2[['gameId', 'actionNumber', 'scoreHome', 'scoreAway', 'actionType']].head(50))

In [ ]:
data_v3_ft["scoreDiff"] = np.where(
    data_v3_ft["location"] == "h",
    data_v3_ft["scoreHome"] - data_v3_ft["scoreAway"],
    data_v3_ft["scoreAway"] - data_v3_ft["scoreHome"]
)

In [ ]:
print(data_v3_ft[['gameId', 'actionNumber', 'scoreHome', 'scoreAway', 'scoreDiff', 'actionType', 'location']].head(50))

In [ ]:
def shift_score_diff(group):
    group = group.copy()
    group['scoreDiff'] = group['scoreDiff'].shift(1)  # Shift up to reflect "before shot"
    group.iloc[0, group.columns.get_loc('scoreDiff')] = 0.0      # First row should be 0
    return group

data_v3_ft = data_v3_ft.groupby('gameId', group_keys=False).apply(shift_score_diff)

def mirror_diff_on_location_switch(group):
    group = group.copy()
    # Compare current row's location to previous row's
    group['locationChanged'] = group['location'] != group['location'].shift(1)
    
    # Mirror scoreDiff only if location changed and scoreDiff != 0
    group['scoreDiff'] = np.where(
        (group['locationChanged']) & (group['scoreDiff'] != 0),
        -group['scoreDiff'],
        group['scoreDiff']
    )
    
    group.drop(columns='locationChanged', inplace=True)  # optional cleanup
    return group

data_v3_ft = data_v3_ft.groupby('gameId', group_keys=False).apply(mirror_diff_on_location_switch)

# print(data_v3_ft[['actionNumber', 'scoreHome', 'scoreAway', 'scoreDiff', 'actionType', 'location']].head(50))

In [ ]:
data_v3_ft.to_csv("data_v3_ft.csv", index=False)

In [17]:
data_v3 = pd.read_csv("data_v3_ft.csv")

data_v3 = data_v3[data_v3["actionType"].isin(["Made Shot", "Missed Shot"])]

In [18]:
data_v3.columns

Index(['gameId', 'actionNumber', 'clock', 'period', 'teamId', 'teamTricode',
       'personId', 'playerNameI', 'xLegacy', 'yLegacy', 'shotDistance',
       'shotResult', 'isFieldGoal', 'scoreHome', 'scoreAway', 'location',
       'actionType', 'subType', 'shotValue', 'actionId', 'FG_PCT', 'FG3_PCT',
       'scoreDiff'],
      dtype='object')

In [19]:
data_v3['shotPct'] = data_v3.apply(
    lambda row: row['FG_PCT'] if row['shotValue'] == 2 else row['FG3_PCT'] if row['shotValue'] == 3 else None,
    axis=1
)

data_v3.drop(columns=["FG_PCT", "FG3_PCT"], inplace=True)

data_v3.head()

,gameId,actionNumber,clock,period,teamId,teamTricode,personId,playerNameI,xLegacy,yLegacy,...,isFieldGoal,scoreHome,scoreAway,location,actionType,subType,shotValue,actionId,scoreDiff,shotPct
0,22400119,7,PT11M55.00S,1,1610612754,IND,1629614,A. Nembhard,-9,32,...,1,2.0,0.0,h,Made Shot,Layup Shot,2,3,0.0,0.469
1,22400119,8,PT11M38.00S,1,1610612738,BOS,201950,J. Holiday,-59,35,...,1,2.0,0.0,v,Missed Shot,Layup Shot,2,4,0.0,0.445
2,22400119,10,PT11M31.00S,1,1610612754,IND,1627783,P. Siakam,-97,76,...,1,2.0,0.0,h,Missed Shot,Floating Jump shot,2,6,-2.0,0.526
3,22400119,12,PT11M22.00S,1,1610612754,IND,1627783,P. Siakam,-174,193,...,1,2.0,0.0,h,Missed Shot,Jump Shot,3,8,-2.0,0.396
4,22400119,14,PT11M07.00S,1,1610612738,BOS,1627759,J. Brown,-224,59,...,1,2.0,0.0,v,Missed Shot,Step Back Jump shot,3,10,-2.0,0.316


In [20]:
data_v3.drop(columns=["gameId", "actionNumber", "teamTricode", "personId", "playerNameI", "scoreHome", "scoreAway", "location", "actionType", "shotValue", "actionId"], inplace=True)

In [21]:
data_v3.head()

,clock,period,teamId,xLegacy,yLegacy,shotDistance,shotResult,isFieldGoal,subType,scoreDiff,shotPct
0,PT11M55.00S,1,1610612754,-9,32,3,Made,1,Layup Shot,0.0,0.469
1,PT11M38.00S,1,1610612738,-59,35,7,Missed,1,Layup Shot,0.0,0.445
2,PT11M31.00S,1,1610612754,-97,76,12,Missed,1,Floating Jump shot,-2.0,0.526
3,PT11M22.00S,1,1610612754,-174,193,26,Missed,1,Jump Shot,-2.0,0.396
4,PT11M07.00S,1,1610612738,-224,59,0,Missed,1,Step Back Jump shot,-2.0,0.316


Data v3 contains scoreDiff and shotPct columns.

In [23]:
data_v3["period"].unique()

array([1, 2, 3, 4, 5])

In [25]:
def clock_to_seconds(clock_str):
    parts = clock_str.replace('PT', '').replace('S', '').split('M')
    minutes = int(parts[0].replace('M', ''))
    seconds = float(parts[1])
    return minutes * 60 + seconds

data_v3['secondsRemaining'] = data_v3['clock'].apply(clock_to_seconds)

# Define clutchFlag
data_v3['clutchFlag'] = data_v3.apply(
    lambda row: 1 if (
        row['period'] in [4, 5] and
        row['secondsRemaining'] <= 300 and
        abs(row['scoreDiff']) <= 5
    ) else 0,
    axis=1
)

data_v3['clutchFlag'].value_counts()

0    818
1     86
Name: clutchFlag, dtype: int64

In [27]:
data_v3.drop(columns=["period", "clock", "secondsRemaining"], inplace=True)
data_v3.columns

In [43]:
data_v3.to_csv("data_v3.csv", index=False)

In [32]:
data_v3 = pd.read_csv("data_v3.csv")

data_v3.drop(columns=["teamId"], inplace=True)

data_v3.head()

,xLegacy,yLegacy,shotDistance,shotResult,isFieldGoal,subType,scoreDiff,shotPct,clutchFlag
0,-9,32,3,Made,1,Layup Shot,0.0,0.469,0
1,-59,35,7,Missed,1,Layup Shot,0.0,0.445,0
2,-97,76,12,Missed,1,Floating Jump shot,-2.0,0.526,0
3,-174,193,26,Missed,1,Jump Shot,-2.0,0.396,0
4,-224,59,0,Missed,1,Step Back Jump shot,-2.0,0.316,0


In [42]:
data_v3["shotResult"] = data_v3["shotResult"].replace({'Made': 1, 'Missed': 0})

data_v3.head(10)

,xLegacy,yLegacy,shotDistance,shotResult,isFieldGoal,subType,scoreDiff,shotPct,clutchFlag
0,-9,32,3,1,1,Layup Shot,0.0,0.469,0
1,-59,35,7,0,1,Layup Shot,0.0,0.445,0
2,-97,76,12,0,1,Floating Jump shot,-2.0,0.526,0
3,-174,193,26,0,1,Jump Shot,-2.0,0.396,0
4,-224,59,0,0,1,Step Back Jump shot,-2.0,0.316,0
5,23,29,4,0,1,Driving Layup Shot,-2.0,0.500,0
6,-10,20,2,1,1,Running Dunk Shot,2.0,0.459,0
7,23,17,3,0,1,Driving Layup Shot,-2.0,0.609,0
8,-25,-1,3,0,1,Running Finger Roll Layup Shot,0.0,0.459,0
9,5,16,2,1,1,Running Layup Shot,0.0,0.609,0
